In [ ]:
# Por si alguien corre en python2
from __future__ import division

# Preparamos todo para correr
import numpy as np
from math import sqrt
from matplotlib import pylab as plt

# *Probabilidades condicionales y el teorema de Bayes*

### Un ejemplo

Una enfermedad genética rara que afecta al 0.1% de la población de un país. Para diagnosticarla, existe un análisis clínico que detecta la enfermedad en el 99% de los casos en los que el paciente la padece, y tiene un 1% de falsos positivos.

Un paciente recibe un resultado positivo del análisis clínico. 

<ul>
    <li>¿Qué probabilidad hay de que la persona padezca la enfermedad?</li>
    <li>¿Cuál sería el próximo paso natural?</li>
</ul>

*****

En primer lugar, pasemos a probabilidades los datos que tenemos:

<ol>
    <li> La probabilidad de tener la enfermedad es 1%; $P(E) = 0.01$</li>
    <li> Si un paciente tiene la enfermedad, el análisis da positivo en el 99% de los casos; $P(D | E) = 0.99$</li>
    <li> El porcentaje de falsos positivos del diagnóstico es 0.1%; $P(D | \bar{E}) = 0.01$</li>
    </ol>
    
Lo que nos preguntamos es cuál es la probabilidad de que el paciente tenga la enfermedad, dado que recibió un resultado positivo. Buscamos entonces $P(E | D)$. <b>¿Cómo se escribe esto en términos de las cantidades que conocemos?</b>

La respuesta se obtiene a partir del teorema de Bayes:

$$
P(E | D) = \frac{P(D | E)\,P(E)}{P(D)}
$$

Muy bien, pero nos falta conocer $P(D)$; para eso, usamos la regla de la suma:

$$
P(D) = P(D | E)\,P(E) + P(D | \bar{E}) P(\bar{E})
$$

Poniendo todo junto:

$$
P(E | D) = \frac{P(D | E)\,P(E)}{P(D | E)\,P(E) + P(D | \bar{E}) P(\bar{E})}
$$

Y ahora sí podemos calcular.

In [ ]:
#Escribo las diferentes probabilidades
pE = 0.001
pDE = 0.99
pDnE = 0.01

pED = pDE * pE / (pDE * pE + pDnE * (1 - pE))

print('La probabilidad de que el paciente tenga la enfermedad es {:.4f}'.format(pED))

***
## Caso: Apruebo si falto a clase?

Tomaremos un set de Datos de _Student Alcohol Consumption_ de la UCI alojada por Kaggle. Pueden descargar el dataset de la clase de matemáticas aquí https://www.kaggle.com/uciml/student-alcohol-consumption#student-mat.csv y subirlo en google Colab, desde la pestaña izquierda. En el repo, ya esta alojado en el directorio _datasets_.

Primero que nada, carguemos el dataset usando pandas:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('datasets/student-alcohol-consumption/student-mat.csv')
print("Cantidad de registros: {}".format(len(df)))
df.head()

Cantidad de registros: 395


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


Los datos son de una encuesta realizada a 395 alumnos de una clase de matemáticas, y tenemos muchos campos. Nos concentraremos en la siguiente pregunta: **¿Cual es la probabilidad de aprobar con una nota de 80% o más si falté 10 o más clases?**

Para hacerlo, crearemos unas columnas con 0 o 1 dependiendo de si cumplen las condiciones o no, así facilitaremos la tarea de contabilizar. La nota final esta en la columna _G3_, en una escala de 0 a 20. Las ausencias se contabilizan en _absences_.

In [5]:
df['aprueba'] = np.where(df['G3']*5 >= 80, 1, 0)
df['falto_mucho'] = np.where(df['absences'] >= 10, 1, 0)

Para contar las columnas, nos será útil crear una columna de _1_ constante, así contar es simplemente sumar dicho campo.

In [6]:
df['cuenta'] = 1

Ahora nos deshacemos de todas las otras columnas:

In [7]:
df = df[['aprueba','falto_mucho','cuenta']]
df.head()

,aprueba,falto_mucho,cuenta
0,0,0,1
1,0,0,1
2,0,1,1
3,0,0,1
4,0,0,1


Lo que sigue es transformar nuestra tabla. Queremos una tabla 2x2, la cuyos ejes sean el valor de "aprueba" y "falto mucho", y cuyos contenidos sea la cantidad de alumnos en cada categoría. Eso lo podemos conseguir utilizando el método `pd.pivot_table` usando la suma como función de agregación de la columna _cuenta_:

In [8]:
pd.pivot_table(
    df,
    values = 'cuenta',
    index = ['aprueba'],
    columns = ['falto_mucho'],
    aggfunc = np.sum,
    fill_value = 0)

falto_mucho,0,1
aprueba,,
0,277,78
1,35,5


Podemos ver entonces que, la cantidad de alumnos que falto poco y desaprobó es 277, que faltó mucho y desaprobó es 78, que faltó poco y aprobó es 35 y que faltó mucho y aprobó es 5, siendo la cantidad total de alumnos 395. Definimos las probabilidades entonces:

$$ P(\text{aprueba}) = \frac{35 + 5}{395} \approx  0.10126582278481013$$ 
$$ P(\text{falto_mucho}) = \frac{78 + 5}{395} \approx 0.21012658227848102$$
$$ P(\text{aprueba} \cap \text{falto_mucho}) = \frac{5}{395} \approx  0.012658227848101266$$

La probabilidad condicional de que hayan aprobado, dado que faltaron mucho, es por definición:
$$  P(\text{aprueba }\, |\, \text{ falto_mucho}) = \frac{P(\text{aprueba} \cap \text{falto_mucho})}{P(\text{falto_mucho})} $$

Es decir:

In [16]:
print("Probabilidad de sacar un 80% o mas dado que falte 10 o mas clases: {:.2f}%".format(5/(78+5)*100))

Probabilidad de sacar un 80% o mas dado que falte 10 o mas clases: 6.02%
